In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot
import pandas as pd
import numpy as np

In [2]:
# Definimos colores fundar
fundar_colores_primarios = ['#7BB5C4', '#9FC1AD', '#D3D3E0',  '#8d9bff', '#FF9750', '#FFD900',]
fundar_colores_secundarios = ['#B5E0EA', '#B3B3B3', '#848279', '#AFA36E', '#5D896F', '#9C9CBC', '#E27124']
fundar_white = '#F5F5F5'
fundar_black = '#151515'

In [3]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'


In [4]:
ratio_actividad = pd.read_csv(pathdata + 'data_output/indicadores/01_ratio_actividad.csv', index_col=0)
empleo_asalariado_con_descuento = pd.read_csv(pathdata + 'data_output/indicadores/02_empleo_asalariado_con_descuento.csv', index_col=0)
horas_remuneradas_media = pd.read_csv(pathdata + 'data_output/indicadores/03_horas_promedio_ocup_ppal.csv', index_col=0)
ingreso_salarial_media = pd.read_csv(pathdata + 'data_output/indicadores/04_ingreso_salarial.csv', index_col=0)
ratio_minutos_promedio_no_pago = pd.read_csv(pathdata + 'data_output/indicadores/05_tiempo_no_remunerado.csv', index_col=0)
inactivos_con_ingreso = pd.read_csv(pathdata + 'data_output/indicadores/06_dependencia_economica.csv', index_col=0)
hogares_jefatura = pd.read_csv(pathdata + 'data_output/indicadores/07_hogares_no-pobres_jefatura.csv', index_col=0)


In [6]:
variables = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, ingreso_salarial_media, ratio_minutos_promedio_no_pago, inactivos_con_ingreso, hogares_jefatura]
columnas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio', 'Tiempo TNR', 'Dependencia', 'No Pobreza']

for columna, variable in zip(columnas,variables):
    variable[columna] = np.where(variable[columna] > 100, 100, variable[columna])


In [7]:
componente_insercion = pd.read_csv(pathdata + 'data_output/componentes_e_indice/01_insercion_laboral.csv', index_col=0)
componente_autonomia_economica = pd.read_csv(pathdata + 'data_output/componentes_e_indice/02_insercion_autonomia_economica.csv', index_col=0)
indice_mti = pd.read_csv(pathdata + 'data_output/componentes_e_indice/01_indice_MTeI.csv', index_col=0)

In [15]:
import plotly.graph_objects as go
from plotly.offline import plot

bar_trace1 = go.Bar(
    x=ratio_actividad.index,
    y=ratio_actividad['Actividad'],
    name='Actividad',
    marker_color=fundar_colores_primarios[0]
)

bar_trace2 = go.Bar(
    x=empleo_asalariado_con_descuento.index,
    y=empleo_asalariado_con_descuento['Formalidad'],
    name='Formalidad',
    marker_color=fundar_colores_primarios[1]
)

bar_trace3 = go.Bar(
    x=horas_remuneradas_media.index,
    y=horas_remuneradas_media['Horas promedio remuneradas'],
    name='Horas promedio remuneradas',
    marker_color=fundar_colores_primarios[2]
)

bar_trace4 = go.Bar(
    x=ingreso_salarial_media.index,
    y=ingreso_salarial_media['Ingreso laboral promedio'],
    name='Ingreso laboral promedio',
    marker_color=fundar_colores_primarios[3]
)

bar_trace5 = go.Bar(
    x=componente_insercion.index,
    y=componente_insercion['Insercion laboral'],
    name='Componente',
    marker_color=fundar_colores_primarios[4]
)

layout = go.Layout(
    title='Indicadores e insercion laboral',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    font_size=20
)

fig = go.Figure(data=[bar_trace1, bar_trace2, bar_trace3, bar_trace4, bar_trace5], layout=layout)

fig.add_shape(
    type='line',
    x0=ratio_actividad.index[0],
    y0=100,
    x1=ratio_actividad.index[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)

file_path = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/'
filename = 'insercion_laboral.html'
plot(fig, filename=file_path + filename, auto_open=False)

'/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/insercion_laboral.html'

In [ ]:
def create_bar_chart(df_list, error_cols, scatter_cols, colors, file_path, filename):
    bar_traces = []
    scatter_traces = []

    for i, df in enumerate(df_list):
        error_y = dict(
            type='data',
            symmetric=False,
            array=df['LS'] - df[error_cols[i]],
            arrayminus=df[error_cols[i]] - df['LI'],
            visible=True,
            color='black',
            thickness=0.7
        )
        
        bar_trace = go.Bar(
            x=df.index,
            y=df[error_cols[i]],
            name='T' + str(i + 1) + '. ' + error_cols[i],
            marker_color=colors[i],
            error_y=error_y
        )
        bar_traces.append(bar_trace)

        scatter_trace = go.Scatter(
            x=df.index,
            y=df[scatter_cols[i]],
            name='T' + str(i + 1) + '. ER',
            mode='markers',
            marker=dict(
                color='black',
                size=8
            ),
            yaxis='y2'
        )
        scatter_traces.append(scatter_trace)

    layout = go.Layout(
        title='Insercion laboral',
        barmode='group',
        plot_bgcolor='rgba(0,0,0,0)',
        legend=dict(y=1.1, orientation='h'),
        yaxis=dict(title='Actividad', range=[0, 170]),
        scattermode="group",
        yaxis2=dict(title='ER', overlaying='y', side='right', range=[0, 70])
    )

    fig = go.Figure(data=bar_traces + scatter_traces, layout=layout)

    fig.add_shape(
        type='line',
        x0=df_list[0].index[0],
        y0=100,
        x1=df_list[0].index[-1],
        y1=100,
        line=dict(
            color='black',
            width=1,
            dash='dash'
        ),
    )

    plot(fig, filename=file_path + filename, auto_open=False)



In [ ]:
def create_componente_insercion_laboral(sorted_data, file_path, filename):
    # Crear la barra de trazado
    bar_trace1 = go.Bar(
        x=sorted_data.index,
        y=sorted_data,
        name=' ',
        marker_color=fundar_colores_primarios[0]
    )

    # Crear el layout
    layout = go.Layout(
        title='Indicador componente insercion laboral',
        barmode='group',
        plot_bgcolor='rgba(0,0,0,0)'
    )

    # Crear la figura
    fig = go.Figure(data=[bar_trace1], layout=layout)

    # Agregar la línea de referencia
    fig.add_shape(
        type='line',
        x0=sorted_data.index[0],
        y0=100,
        x1=sorted_data.index[-1],
        y1=100,
        line=dict(
            color='black',
            width=1,
            dash='dash'
        ),
    )

    fig.update_yaxes(range=[0, 90])

    # Guardar el gráfico como un archivo HTML
    plot(fig, filename=file_path + filename, auto_open=False)


In [ ]:
def create_stacked_bar_chart(df_list, y_columns, colors, title, file_path, filename):
    bar_traces = []

    for i, df in enumerate(df_list):
        x = df.index
        for j, y_col in enumerate(y_columns):
            y_values = df[y_col]
            
            bar_trace = go.Bar(
                x=x,
                y=y_values,
                name=f'{y_col}',
                marker_color=colors[i]
            )
            bar_traces.append(bar_trace)

    layout = go.Layout(
        title=title,
        barmode='group',
        plot_bgcolor='rgba(0,0,0,0)',
        legend=dict(y=1.1, orientation='h'),
        yaxis=dict(title='Cantidad'),
        xaxis=dict(title='Provincia'),
    )

    fig = go.Figure(data=bar_traces, layout=layout)

    plot(fig, filename=file_path + filename, auto_open=False)


In [ ]:
file_path = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/'

sorted_data = insercion_laboral.sort_values(ascending=False)

filename = 'componente_insercion_laboral_16-a-65.html'
create_componente_insercion_laboral(sorted_data, file_path, filename)

df_list = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media]

y_columns = ['Varon', 'Mujer']
colors = [fundar_colores_primarios[0], 
          fundar_colores_primarios[1],
          fundar_colores_primarios[2],
          fundar_colores_primarios[3],
          fundar_colores_primarios[4]
          ]

title = 'Comparación de Varones y Mujeres por Provincia' 
filename = 'tasas_indicadores_insercion_laboral_16-a-65.html'

create_stacked_bar_chart(df_list, y_columns, colors, title, file_path, filename)

df_list = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media]
error_cols = ['Actividad', 'Acceso a derechos laborales', 'Participacion trabajo remunerado']
scatter_cols = ['ER'] * len(df_list)
colors = fundar_colores_primarios[:len(df_list)]
filename = 'insercion_laboral_16-a-65.html'

create_bar_chart(df_list, error_cols, scatter_cols, colors, file_path, filename)